In [94]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import expm
from scipy.optimize import minimize

### The Adjoint Representation

A Lie algebra is a set of operators (our basis $\{A_p\}$) that is closed under a binary operation called the Lie bracket, which for our purposes is the commutator $[A_i, A_j] = A_iA_j - A_jA_i$.

The "closure" property means that the commutator of any two operators in the basis can be written as a linear combination of other operators within that same basis. We can express this formally with **structure constants**, denoted by $\gamma_{k}^{ij}$:

$$
[A_i, A_j] = \sum_{k=1}^{N} \gamma_{k}^{ij} A_k
$$

Here, $N$ is the dimension of the algebra (in our case, $N=5$). The structure constants uniquely define the algebra.

The **adjoint representation** is a way to map each abstract operator $A_i$ to a concrete $N \times N$ matrix, which we'll call $M_i$. This mapping is defined directly by the structure constants. The element in the $k$-th row and $j$-th column of the matrix $M_i$ is given by:

$$
(M_{i})_{k,j} = \gamma_{k}^{ij}
$$

This definition has a very practical interpretation: The $j$-th column of the matrix $M_i$ is simply the list of coefficients of the basis operators $\{A_k\}$ in the expansion of the commutator $[A_i, A_j]$.

To generate these matrices programmatically, we first need to encode the commutation rules of our algebra. We can represent the basis operators $A_1, \dots, A_5$ and store their commutation relations in a dictionary. The result of each commutator is a list of coefficients corresponding to the basis $(A_1, A_2, A_3, A_4, A_5)$.

For example, the rule $[A_1, A_2] = A_3$ is encoded as the coefficient vector `[0, 0, 1, 0, 0]`. The rule $[A_1, A_4] = -A_3$ is encoded as `[0, 0, -1, 0, 0]`.


In [ ]:
# Helper function to set commutators, automatically handling antisymmetry
def set_commutator(commutator_dict:dict, i:int, j:int, result_vector:list):
    """Populates a dictionary with commutator relations."""
    # Initialize nested dictionaries if they don't exist
    if i not in commutator_dict:
        commutator_dict[i] = {}
    if j not in commutator_dict:
        commutator_dict[j] = {}
        
    # [A_i, A_j] = result
    commutator_dict[i][j] = np.array(result_vector)
    # [A_j, A_i] = -result
    commutator_dict[j][i] = -np.array(result_vector)


def generate_adjoint_matrices(commutators:dict, num_operators:int):
    """
    Generates the adjoint matrices for a given Lie algebra.

    Args:
        commutators (dict): A dictionary defining the commutation relations.
                            commutators[i][j] gives the result of [A_{i+1}, A_{j+1}].
        num_operators (int): The dimension of the Lie algebra.

    Returns:
        list: A list of the generated NxN adjoint matrices.
    """
    basis_indices = range(num_operators)
    adjoint_matrices = []
    
    for i in basis_indices:  # For each matrix M_i we want to build
        M_i = np.zeros((num_operators, num_operators))
        for j in basis_indices:  # For each column j of M_i
            # The j-th column of M_i is the result of [A_i, A_j]
            # Check if the commutator is defined, otherwise it's zero
            if j in commutators.get(i, {}):
                result_vector = commutators[i][j]
                M_i[:, j] = result_vector
        adjoint_matrices.append(M_i)
        
    return adjoint_matrices

### The Seniority-2 Lie Algebra

The seniority-2 case involves a 5-element Lie algebra constructed from two initial generator operators, $T_a$ and $T_b$. The entire algebra is built from these generators and their nested commutators.

#### Basis Operators

The five basis operators of the algebra, denoted as $\{A_p\}$, are defined as follows:

1.  $A_1 = T_a$
2.  $A_2 = T_b$
3.  $A_3 = [T_a, T_b]$
4.  $A_4 = [T_a, [T_a, T_b]] = [A_1, A_3]$
5.  $A_5 = [T_b, [T_b, T_a]] = -[T_b, [T_a, T_b]] = -[A_2, A_3]$

The set $\{A_1, A_2, A_3, A_4, A_5\}$ is closed under the commutation operation, meaning the commutator of any two operators in the set results in a linear combination of other operators from the same set.

#### Commutation Relations

The structure of the algebra is defined by the following fundamental commutation relations. These relations are derived from the operator definitions and the identities provided in the source document.

* **Commutators that generate the basis:**
    $$[A_1, A_2] = A_3$$
    $$[A_1, A_3] = A_4$$
    $$[A_2, A_3] = -A_5$$

* **Commutators that show the algebra's cyclic nature:**
    $$[A_1, A_4] = -A_3$$
    $$[A_2, A_5] = A_3$$

* **Commutators between higher-order operators:**
    $$[A_3, A_4] = -A_5$$
    $$[A_3, A_5] = A_4$$
    $$[A_4, A_5] = A_3$$

* **Commutators that result in zero:**
    $$[A_1, A_5] = 0$$
    $$[A_2, A_4] = 0$$


In [73]:
num_ops = 5
seniority2_algebra = {}


# [A1, A2] = A3
set_commutator(seniority2_algebra, 0, 1, [0, 0, 1, 0, 0])
# [A1, A3] = A4
set_commutator(seniority2_algebra, 0, 2, [0, 0, 0, 1, 0])
# [A1, A4] = -A3
set_commutator(seniority2_algebra, 0, 3, [0, 0, -1, 0, 0])
# [A2, A3] = -A5
set_commutator(seniority2_algebra, 1, 2, [0, 0, 0, 0, -1])
# [A2, A5] = A3
set_commutator(seniority2_algebra, 1, 4, [0, 0, 1, 0, 0])
# [A3, A4] = -A5
set_commutator(seniority2_algebra, 2, 3, [0, 0, 0, 0, -1])
# [A3, A5] = A4
set_commutator(seniority2_algebra, 2, 4, [0, 0, 0, 1, 0])
# [A4, A5] = A3
set_commutator(seniority2_algebra, 3, 4, [0, 0, 1, 0, 0])



adjoint_matrices = generate_adjoint_matrices(seniority2_algebra, num_operators=num_ops)


print("--- Adjoint Matrices ---\n")
for i, M in enumerate(adjoint_matrices):
    print(f"--- M_{i+1} ---")
    print(M)
    print("-" * 15 + "\n")

--- Adjoint Matrices ---

--- M_1 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
---------------

--- M_2 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]]
---------------

--- M_3 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.]
 [ 0.  1.  0. -1.  0.]]
---------------

--- M_4 ---
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
---------------

--- M_5 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
---------------



In [ ]:
def solve_decomposition(adjoint_matrices:list ,theta_coeffs:list, num_operators:int):
    """
    Solves the Lie algebra decomposition problem for a given theta.

    e^{sum_i theta_i * A_i} = product_p e^{t_p(theta) * A_p}

    This function uses the adjoint matrix representation to find the
    parameters t_p(theta).

    Args:
        theta (list): The list of parameters for the operator exponential.

    Returns:
        scipy.optimize.OptimizeResult: The result from the optimizer,
                                       containing the solution for t_p.
    """

    # 2. Compute the Left-Hand Side (The Target Matrix)
    M_sum = np.zeros((num_operators, num_operators))

    for i in range(num_operators):
        M_sum += theta_coeffs[i] * adjoint_matrices[i]
    U_target = expm(M_sum)
    
    # 3. Define the Cost Function for Numerical Optimization
    def cost_function(t_params):
        """
        Calculates the difference between the target matrix and the product.

        Args:
            t_params (list or np.array): The list of five parameters [t1, t2, t3, t4, t5].

        Returns:
            float: The squared Frobenius norm of the difference, which the
                   optimizer will try to minimize.
        """
        # Calculate the product on the right-hand side:
        # U_prod = exp(t1*M1) @ exp(t2*M2) @ ... @ exp(t5*M5)
        # Note: The '@' operator is used for matrix multiplication.
        U_prod = np.identity(num_operators) # Start with identity matrix
        for i in range(num_operators):
            U_prod = U_prod @ expm(t_params[i] * adjoint_matrices[i])

        # Calculate the error (cost)
        error_matrix = U_prod - U_target
        cost = norm(error_matrix, ord=2)
        return cost

    # 4. Perform the Numerical Optimization
    # We need an initial guess for the parameters t_p.
    # A simple guess is to start with all zeros.

    initial_guess = np.zeros(num_operators)
    # initial_guess = theta_coeffs

    # Use the L-BFGS-B optimizer to find the t_p that minimize the cost function.
    result = minimize(cost_function, initial_guess, method='L-BFGS-B', tol=1e-50)

    return result, U_target, adjoint_matrices

In [ ]:
# THETA_COEFFS = [0.200, 0.300, 0.100, -0.100, 0.050]
THETA_COEFFS = [0.5, 0.5, 0, 0, 0]
print(f"--- Solving for theta_coeffs = {THETA_COEFFS} ---\n")

# Solve the decomposition
optimization_result, U_target, matrices = solve_decomposition(adjoint_matrices, THETA_COEFFS, num_ops)

# Extract the solution
t_solution = optimization_result.x

# Print the results
if optimization_result.success:
    print("Optimization successful!")
    print(f"Found solution for t_p:")
    for i in range(5):
        print(f"  t_{i+1} = {t_solution[i]:.8f}")
    print(f"\nFinal cost (error): {optimization_result.fun:.2e}\n")

else:
    print("Optimization failed.")
    print("Message:", optimization_result.message)

# 5. Final Verification
print("--- Verification ---")
print("Comparing the target matrix with the matrix from the solution.\n")

# Calculate the product matrix using the found solution
U_solution = np.identity(num_ops)
for i in range(num_ops):
    U_solution = U_solution @ expm(t_solution[i] * matrices[i])

print("Target Matrix (LHS):")
print(np.round(U_target, 4))
print("\nProduct Matrix from Solution (RHS):")
print(np.round(U_solution, 4))

# Calculate the final difference
target_norm = norm(U_target, ord=2)
final_difference = norm(U_solution - U_target, ord=2)
print(f"\nSpectral norm of the target: {target_norm:.2e}")
print(f"\nSpectral norm of the difference: {final_difference:.2e}")

--- Solving for theta_coeffs = [0.2, 0.3, 0.1, -0.1, 0.05] ---

Optimization successful!
Found solution for t_p:
  t_1 = 0.18795636
  t_2 = 0.29967742
  t_3 = 0.06762692
  t_4 = -0.09750264
  t_5 = 0.03605055

Final cost (error): 3.41e-02

--- Verification ---
Comparing the target matrix with the matrix from the solution.

Target Matrix (LHS):
[[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [-0.3796  0.1566  0.943  -0.2547  0.1835]
 [-0.1263  0.0156  0.1271  0.9777  0.1128]
 [ 0.0834  0.0685 -0.3991 -0.0484  0.9566]]

Product Matrix from Solution (RHS):
[[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [-0.3671  0.1729  0.9241 -0.2392  0.1864]
 [-0.1383  0.0182  0.1324  0.971   0.1046]
 [ 0.0801  0.0786 -0.3876 -0.0506  0.9866]]

Spectral norm of the target: 1.26e+00

Spectral norm of the difference: 3.41e-02


### Seniority 4 WorkInProgress

In [95]:
def populate_seniority4_algebra():
    """
    Defines the basis and populates the commutation relations for the 29-element
    seniority-4 Lie algebra provided in the document.

    This function systematically translates every identity from the document
    into the commutator dictionary.

    Returns:
        dict: A dictionary defining the commutation relations.
        dict: A mapping from operator names to their integer index.
        list: A list of the operator names.
    """
    # 1. Define the 29 basis operators
    # The first 28 are the unique, independent nested commutators found in the
    # provided identities. The 29th is the Identity operator.
    op_names = [
        # Level 0: Generators (4 ops)
        'Ta', 'Tb', 'Tc', 'Td',
        # Level 1: Commutators of Level 0 (4 new ops)
        '[Ta,Tc]', '[Ta,Td]', '[Tb,Tc]', '[Tb,Td]',
        # Level 2: Commutators of Level 0 and 1 (12 new ops)
        '[Ta,[Ta,Tc]]', '[Ta,[Ta,Td]]', '[Ta,[Tb,Tc]]', '[Ta,[Tb,Td]]',
        '[Tb,[Tb,Tc]]', '[Tb,[Tb,Td]]',
        '[Tc,[Ta,Tc]]', '[Tc,[Ta,Td]]', '[Tc,[Tb,Tc]]', '[Tc,[Tb,Td]]',
        '[Td,[Ta,Td]]', '[Td,[Tb,Td]]',
        # Level 3: Commutators of Level 0 and 2 (4 new ops)
        '[Ta,[Ta,[Tb,Tc]]]', '[Ta,[Ta,[Tb,Td]]]',
        '[Ta,[Tb,[Tb,Tc]]]', '[Ta,[Tb,[Tb,Td]]]',
        # Level 4: Commutators of Level 0 and 3 (2 new ops)
        '[Ta,[Ta,[Tb,[Tb,Tc]]]]', '[Ta,[Ta,[Tb,[Tb,Td]]]]',
        # Level "C": Commutators involving Tc (2 new ops)
        '[Tc,[Ta,[Ta,[Tb,Tc]]]]', '[Tc,[Ta,[Tb,[Tb,Tc]]]]',
        # The Identity Operator
        'Id'
    ]
    
    num_operators = len(op_names)
    op_to_idx = {name: i for i, name in enumerate(op_names)}

    # Initialize the dictionary to store commutation relations
    commutators = {}

    # Helper function to set commutators, automatically handling antisymmetry
    def set_commutator(i_name, j_name, result_tuple):
        i_idx = op_to_idx[i_name]
        j_idx = op_to_idx[j_name]
        
        if i_idx not in commutators: commutators[i_idx] = {}
        if j_idx not in commutators: commutators[j_idx] = {}
        
        vec = np.zeros(num_operators)
        if result_tuple:  # result_tuple is (coefficient, result_name)
            coeff, result_name = result_tuple
            vec[op_to_idx[result_name]] = coeff

        commutators[i_idx][j_idx] = vec
        commutators[j_idx][i_idx] = -vec

    # 2. Populate the dictionary based on ALL provided relations
    
    # --- Zeroth Order Commutators ---
    set_commutator('Ta', 'Tb', None)
    set_commutator('Tc', 'Td', None)

    # --- Jacobi-like Identities ---
    set_commutator('Tb', '[Ta,Tc]', (1.0, '[Ta,[Tb,Tc]]'))
    set_commutator('Tb', '[Ta,Td]', (1.0, '[Ta,[Tb,Td]]'))
    set_commutator('Td', '[Ta,Tc]', (1.0, '[Tc,[Ta,Td]]'))
    set_commutator('Td', '[Tb,Tc]', (1.0, '[Tc,[Tb,Td]]'))

    # --- Triple Commutator Identities ---
    set_commutator('Ta', '[Ta,[Ta,Tc]]', (-1.0, '[Ta,Tc]'))
    set_commutator('Ta', '[Ta,[Ta,Td]]', (-1.0, '[Ta,Td]'))
    set_commutator('Tb', '[Tb,[Tb,Tc]]', (-1.0, '[Tb,Tc]'))
    set_commutator('Tb', '[Tb,[Tb,Td]]', (-1.0, '[Tb,Td]'))
    set_commutator('Tc', '[Tc,[Ta,Tc]]', (-1.0, '[Ta,Tc]'))
    set_commutator('Tc', '[Tc,[Tb,Tc]]', (-1.0, '[Tb,Tc]'))
    set_commutator('Td', '[Td,[Ta,Td]]', (-1.0, '[Ta,Td]'))
    set_commutator('Td', '[Td,[Tb,Td]]', (-1.0, '[Tb,Td]'))

    # --- Zero Commutators ---
    set_commutator('Ta', '[Tc,[Ta,Tc]]', None)
    set_commutator('Ta', '[Tc,[Ta,Td]]', None)
    set_commutator('Ta', '[Tc,[Tb,Tc]]', None)
    set_commutator('Ta', '[Tc,[Tb,Td]]', None)
    set_commutator('Ta', '[Td,[Ta,Td]]', None)
    set_commutator('Ta', '[Td,[Tb,Td]]', None)
    set_commutator('Tb', '[Tc,[Ta,Tc]]', None)
    set_commutator('Tb', '[Tc,[Ta,Td]]', None)
    set_commutator('Tb', '[Tc,[Tb,Tc]]', None)
    set_commutator('Tb', '[Tc,[Tb,Td]]', None)
    set_commutator('Tb', '[Td,[Ta,Td]]', None)
    set_commutator('Tb', '[Td,[Tb,Td]]', None)
    set_commutator('Tc', '[Ta,[Ta,Tc]]', None)
    set_commutator('Tc', '[Ta,[Ta,Td]]', None)
    set_commutator('Tc', '[Ta,[Tb,Tc]]', None)
    set_commutator('Tc', '[Ta,[Tb,Td]]', None)
    set_commutator('Tc', '[Tb,[Tb,Tc]]', None)
    set_commutator('Tc', '[Tb,[Tb,Td]]', None)
    set_commutator('Td', '[Ta,[Ta,Tc]]', None)
    set_commutator('Td', '[Ta,[Ta,Td]]', None)
    set_commutator('Td', '[Ta,[Tb,Tc]]', None)
    set_commutator('Td', '[Ta,[Tb,Td]]', None)
    set_commutator('Td', '[Tb,[Tb,Tc]]', None)
    set_commutator('Td', '[Tb,[Tb,Td]]', None)

    # --- More Jacobi-like Identities ---
    set_commutator('Tb', '[Ta,[Ta,Tc]]', (1.0, '[Ta,[Ta,[Tb,Tc]]]'))
    set_commutator('Tb', '[Ta,[Ta,Td]]', (1.0, '[Ta,[Ta,[Tb,Td]]]'))
    set_commutator('Tb', '[Ta,[Tb,Tc]]', (1.0, '[Ta,[Tb,[Tb,Tc]]]'))
    set_commutator('Tb', '[Ta,[Tb,Td]]', (1.0, '[Ta,[Tb,[Tb,Td]]]'))

    # --- Mixed Identities ---
    set_commutator('Tc', '[Tc,[Ta,Td]]', (1.0, '[Ta,[Tb,[Tb,Td]]]'))
    set_commutator('Tc', '[Tc,[Tb,Td]]', (1.0, '[Ta,[Ta,[Tb,Td]]]'))
    set_commutator('Tc', '[Td,[Ta,Td]]', (1.0, '[Ta,[Tb,[Tb,Tc]]]'))
    set_commutator('Tc', '[Td,[Tb,Td]]', (1.0, '[Ta,[Ta,[Tb,Tc]]]'))
    set_commutator('Td', '[Tc,[Ta,Tc]]', (1.0, '[Ta,[Tb,[Tb,Td]]]'))
    set_commutator('Td', '[Tc,[Ta,Td]]', (1.0, '[Ta,[Tb,[Tb,Tc]]]'))
    set_commutator('Td', '[Tc,[Tb,Tc]]', (1.0, '[Ta,[Ta,[Tb,Td]]]'))
    set_commutator('Td', '[Tc,[Tb,Td]]', (1.0, '[Ta,[Ta,[Tb,Tc]]]'))

    # --- Quadruple Commutator Identities ---
    set_commutator('Ta', '[Ta,[Ta,[Tb,Tc]]]', (-1.0, '[Ta,[Tb,Tc]]'))
    set_commutator('Ta', '[Ta,[Ta,[Tb,Td]]]', (-1.0, '[Ta,[Tb,Td]]'))
    set_commutator('Tb', '[Ta,[Ta,[Tb,Tc]]]', (1.0, '[Ta,[Ta,[Tb,[Tb,Tc]]]]'))
    set_commutator('Tb', '[Ta,[Ta,[Tb,Td]]]', (1.0, '[Ta,[Ta,[Tb,[Tb,Td]]]]'))
    set_commutator('Tb', '[Ta,[Tb,[Tb,Tc]]]', (-1.0, '[Ta,[Tb,Tc]]'))
    set_commutator('Tb', '[Ta,[Tb,[Tb,Td]]]', (-1.0, '[Ta,[Tb,Td]]'))
    set_commutator('Tc', '[Ta,[Ta,[Tb,Td]]]', (-1.0, '[Tc,[Tb,Td]]')) # Note: This is [Tc,A21] = -A17
    set_commutator('Td', '[Ta,[Ta,[Tb,Tc]]]', (-1.0, '[Tc,[Tb,Td]]')) # Note: This is [Td,A20] = -A17
    set_commutator('Td', '[Ta,[Ta,[Tb,Td]]]', (1.0, '[Tc,[Ta,[Ta,[Tb,Tc]]]]'))
    set_commutator('Td', '[Ta,[Tb,[Tb,Tc]]]', (-1.0, '[Tc,[Ta,Td]]'))
    set_commutator('Td', '[Ta,[Tb,[Tb,Td]]]', (1.0, '[Tc,[Ta,[Tb,[Tb,Tc]]]]'))

    # --- Quintuple Commutator Identities ---
    set_commutator('Ta', '[Ta,[Ta,[Tb,[Tb,Tc]]]]', (-1.0, '[Ta,[Tb,[Tb,Tc]]]'))
    set_commutator('Ta', '[Ta,[Ta,[Tb,[Tb,Td]]]]', (-1.0, '[Ta,[Tb,[Tb,Td]]]'))
    set_commutator('Ta', '[Tc,[Ta,[Ta,[Tb,Tc]]]]', None)
    set_commutator('Ta', '[Tc,[Ta,[Tb,[Tb,Tc]]]]', None)
    set_commutator('Tb', '[Ta,[Ta,[Tb,[Tb,Tc]]]]', (-1.0, '[Ta,[Ta,[Tb,Tc]]]'))
    set_commutator('Tb', '[Ta,[Ta,[Tb,[Tb,Td]]]]', (-1.0, '[Ta,[Ta,[Tb,Td]]]'))
    set_commutator('Tb', '[Tc,[Ta,[Ta,[Tb,Tc]]]]', None)
    set_commutator('Tb', '[Tc,[Ta,[Tb,[Tb,Tc]]]]', None)
    set_commutator('Tc', '[Ta,[Ta,[Tb,[Tb,Tc]]]]', None)
    set_commutator('Tc', '[Ta,[Ta,[Tb,[Tb,Td]]]]', None)
    set_commutator('Tc', '[Tc,[Ta,[Ta,[Tb,Tc]]]]', (-1.0, '[Ta,[Ta,[Tb,Tc]]]'))
    set_commutator('Tc', '[Tc,[Ta,[Tb,[Tb,Tc]]]]', (-1.0, '[Ta,[Tb,[Tb,Tc]]]'))
    set_commutator('Td', '[Ta,[Ta,[Tb,[Tb,Tc]]]]', None)
    set_commutator('Td', '[Ta,[Ta,[Tb,[Tb,Td]]]]', None)
    set_commutator('Td', '[Tc,[Ta,[Ta,[Tb,Tc]]]]', (-1.0, '[Ta,[Ta,[Tb,Td]]]'))
    set_commutator('Td', '[Tc,[Ta,[Tb,[Tb,Tc]]]]', (-1.0, '[Ta,[Tb,[Tb,Td]]]'))

    # --- Commutators with Identity are zero ---
    id_idx = op_to_idx['Id']
    for i in range(num_operators):
        # set_commutator handles both [A,Id] and [Id,A]
        set_commutator(op_names[i], 'Id', None)

    return commutators, op_to_idx, op_names
    

In [97]:
s4_algebra, s4_op_map, s4_op_names = populate_seniority4_algebra()

print(f"Successfully defined an algebra with {len(s4_op_names)} operators.")
print("\n--- Verification of a few relations ---")

def verify_comm(op1_name, op2_name):
    idx1 = s4_op_map[op1_name]
    idx2 = s4_op_map[op2_name]
    
    if idx1 not in s4_algebra or idx2 not in s4_algebra[idx1]:
        print(f"Commutator [{op1_name}, {op2_name}] is not explicitly defined (implies zero).")
        return

    result_vector = s4_algebra[idx1][idx2]
    non_zero_indices = np.where(result_vector != 0)[0]
    
    if len(non_zero_indices) == 0:
        print(f"[{op1_name}, {op2_name}] = 0")
    else:
        result_str = " + ".join([f"({result_vector[i]:.1f} * {s4_op_names[i]})" for i in non_zero_indices])
        print(f"[{op1_name}, {op2_name}] = {result_str}")

verify_comm('Ta', 'Tc')
verify_comm('Ta', '[Ta,[Ta,Tc]]')
verify_comm('Td', '[Tc,[Ta,[Ta,[Tb,Tc]]]]')
verify_comm('Tc', '[Tc,[Ta,Td]]')
verify_comm('Ta', 'Id')

Successfully defined an algebra with 29 operators.

--- Verification of a few relations ---
Commutator [Ta, Tc] is not explicitly defined (implies zero).
[Ta, [Ta,[Ta,Tc]]] = (-1.0 * [Ta,Tc])
[Td, [Tc,[Ta,[Ta,[Tb,Tc]]]]] = (-1.0 * [Ta,[Ta,[Tb,Td]]])
[Tc, [Tc,[Ta,Td]]] = (1.0 * [Ta,[Tb,[Tb,Td]]])
[Ta, Id] = 0
